# Pipeline 
- Insertar Imagen
- Procesar Imagen
- Separar en líneas
- Enviar a los distintos OCRs
- Correcciones 
- Decisión Final

## Insertar Imágenes

In [1]:
import os


input_folder = "data/input"
os.makedirs(input_folder, exist_ok=True)
output_folder = "data/output_preprocessing"
os.makedirs(output_folder, exist_ok=True)

# Procesar Imágenes

In [2]:
from Preprocessing.pipeline2 import cleanup_image

cleanup_image(input_folder, output_folder)

Procesamiento completo.


## Separar en líneas

## Enviar a los distintos OCRs

In [3]:
import os
from typing import List
from Recognition.Kraken.kraken_model import get_transcription

processed_photos = os.listdir(output_folder)

transcriptions = []

for photo in processed_photos:
    transcriptions.append(get_transcription(output_folder + '/' + photo))
    
# output_ocr = 'data/output_ocr'
# transcriptions = os.listdir(output_ocr)
# results_from_ocrs:List[List[List[str]]] = []

ocr_results = []

# Para cada imagen tenemos una transcripcion
for image in transcriptions:
    lines = []
    for model in image:         # La transcripcion viene del modo model: x text: y
        text = model['text']
        lines = text.split('\n')        # Separamos el texto por lineas
    ocr_results.append(lines)


c:\Users\53552\Desktop\Comp. Science\4to año\Machine Learning\Proyecto Kevin\Actas-Capitulares-de-la-Habana\Recognition\SimpleHTR\SimpleHTR\src\model.py:73: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  conv_norm = tf.compat.v1.layers.batch_normalization(conv, training=self.is_train)



Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\Users\53552\Desktop\Comp. Science\4to año\Machine Learning\Proyecto Kevin\Actas-Capitulares-de-la-Habana\Recognition\SimpleHTR\SimpleHTR\src\model.py:86: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cells = [tf.compat.v1.nn.rnn_cell.LSTMCell(num_units=num_hidden, state_is_tuple=True) for _ in


Python: 3.11.5 (tags/v3.11.5:cce6ba9, Aug 24 2023, 14:38:34) [MSC v.1936 64 bit (AMD64)]
Tensorflow: 2.15.1
Init with stored values from Recognition/SimpleHTR/SimpleHTR/model/snapshot-13
INFO:tensorflow:Restoring parameters from Recognition/SimpleHTR/SimpleHTR/model/snapshot-13
Ejecutando OCR con el modelo bdd-wormser-scriptorium-abbreviated-0.2 sobre la imagen data/output_preprocessing/binarized.png
Ejecutando OCR con el modelo McCATMuS_nfd_nofix_V1 sobre la imagen data/output_preprocessing/binarized.png
OCR completado para todas las imágenes segmentadas.


## Correcciones

In [5]:
from serialization.serialization import correction, combine, load_corrector_model
import google.generativeai as genai
from dotenv import load_dotenv

# results_from_ocr = [
#     [
#         "Essta es una prueva de ectraccion de teexto. Connoscida cosa sea a todos los queesta carta uieren como yo don fffffff por la gracia de dios hey de Castiella.",
#         "Asta carta fue escrita en el anno del senyor de 1493, y  cosas de gran ymportancia para los reynos."
#     ],
#     [
#         "Essa s una prsueva de etraccion de . Connoda cosa sea a tdos as los qeesta carta ueren como yo don Fferrando por la gracia de d hey de Castiella.",
#         "Asta s fue escrita en el ano de seyor de 143, y consigna cossdas de gran ymportana para los reynos."
#     ],
# ]

# Cargar claves de entorno para Gemini
load_dotenv()
GENAI_API_KEY = os.getenv("GENAI_API_KEY")
genai.configure(api_key=GENAI_API_KEY)

spacy_model = "es_core_news_sm"
frequency_dictionary_path = "serialization/spanish_frequency_dictionary.txt"

# Cargar modelo SymSpell y Spacy
nlp, sym_spell = load_corrector_model(
    spacy_model=spacy_model,
    frequency_dictionary_path=frequency_dictionary_path,
    max_edit_distance=3,
    prefix_length=7
)

gemini_model_name="gemini-2.0-flash-exp"

# Crear el sistema de instrucciones para Gemini
chat = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction=[
        """
        Eres un asistente que ayuda a refinar textos. Analiza el siguiente texto original y corregido,
        y reescríbelo en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
        extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
        cartas en Cuba de los siglos XV y XVI, manten el español antiguo.
        """
    ]
).start_chat(history=[])


OSError: [E050] Can't find model 'es_core_news_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

In [2]:
results_from_corrections: List[List[str]] = []

for document in ocr_results:
    aux: List[str] = []
    for line in document:
        aux.append(correction(line, nlp, sym_spell, chat))
    results_from_corrections.append(aux)
    
print(results_from_corrections)

[['Esta es una prueba de extracción de texto. Conocida cosa sea a todos los que esta carta vieren, como yo, don [nombre ilegible], por la gracia de Dios, [rey/señor] de Castilla.', 'Esta carta fue escrita en el año del Señor de 1493, y trata de cosas de gran importancia para los reinos.'], ['Esta es una prueba de extracción. Conocida cosa sea a todos los que esta carta vieren, como yo, don Ferrando, por la gracia de Dios, [rey/señor] de Castilla.', 'Esta fue escrita en el año del Señor de 1493, y consigna cosas de gran importancia para los reinos.']]


## Decisión Final

In [3]:
result:List[str] = []

# Crear el modelo generativo y configurar el sistema de instrucciones
chat2 = genai.GenerativeModel(
    model_name=gemini_model_name,
    system_instruction="""
    Eres un asistente que ayuda a refinar textos. Analiza las siguientes representaciones de una línea de texto,
    y reescribe solo una en un formato limpio, coherente y gramaticalmente correcto. Los textos son documentos históricos
    extraídos mediante OCR, por lo que pueden contener errores típicos del procesamiento OCR. Los documentos son
    cartas en Cuba de los siglos XV y XVI, manten el español antiguo en tu respuesta.
    """
).start_chat(history=[])

for it in range(len(ocr_results[0])):
    aux: List[str] = []
    for i in range(len(ocr_results)):
        aux.append(results_from_corrections[i][it])
    
    result.append(combine(aux, chat2))

print("RESULT FINAL")
print(result)

RESULT FINAL
['Conocida cosa sea a todos los que esta carta vieren, como yo, don Ferrando, por la gracia de Dios, [rey/señor] de Castilla.', 'Esta fue escrita en el año del Señor de 1493, y consigna cosas de gran importancia para los reinos.']
